In [11]:
from alpha_vantage.timeseries import TimeSeries
import requests

In [23]:
api_key = '.....'
symbol = 'AAPL'  # Symbol for the company you want to retrieve data for

In [85]:
ebidta_multiple = 21
annual_growth = 0.07
tax_rate = 0.21
discount_rate = 0.10

In [35]:
# Make a request to the Alpha Vantage API
url_income_statement = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
response_is = requests.get(url_income_statement)
data_is = response_is.json()

url_balance_sheet = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}'
response_bs = requests.get(url_balance_sheet)
data_bs = response_bs.json()

url_cash_flow = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={api_key}'
response_cf = requests.get(url_cash_flow)
data_cf = response_cf.json()

In [80]:
# Extract the data
income_statements = data_is['annualReports']
balance_sheets = data_bs['annualReports']
cash_flows = data_cf['annualReports']
ebidta_0 = int(income_statements[0]['ebitda'])
ebit_0 = int(income_statements[0]['ebit'])
afterTax_0 = ebit_0 * (1 - tax_rate)
capex_0 = int(cash_flows[0]['capitalExpenditures'])
nwc_0 = int(balance_sheets[0]['totalAssets']) - int(balance_sheets[0]['totalLiabilities'])
fcf_0 = afterTax_0 - capex_0 - nwc_0
presentValue_0 = fcf_0
totalDebt_0 = int(balance_sheets[0]['totalLiabilities'])
cash_0 = int(balance_sheets[0]['cashAndCashEquivalentsAtCarryingValue'])
netDebt_0 = totalDebt_0 - cash_0
sharesOutstanding_0 = int(balance_sheets[0]['commonStockSharesOutstanding'])

In [81]:
# EBIDTA
ebidta_values = [ebidta_0]  # Create a list with the initial value
for i in range(1, 6):  # Generate five additional values
    ebidta_i = ebidta_values[i - 1] * (annual_growth + 1)
    ebidta_values.append(ebidta_i)

# EBIT
ebit_values = [ebit_0]  # Create a list with the initial value
for i in range(1, 6):  # Generate five additional values
    ebit_i = ebit_values[i - 1] * (annual_growth + 1)
    ebit_values.append(ebit_i)
    
# After Tax
afterTax_values = [afterTax_0]  # Create a list with the initial value
for i in range(1, 6):  # Generate five additional values
    afterTax_i = ebit_values[i] * (1 - tax_rate)
    afterTax_values.append(afterTax_i)

# Net Working Capital
nwc_values = [nwc_0]  # Create a list with the initial value
for i in range(1, 6):  # Generate five additional values
    nwc_i = nwc_values[i - 1] * (annual_growth + 1)
    nwc_values.append(nwc_i)
    
# Capital Expenditure
capex_values = [capex_0, capex_0, capex_0, capex_0, capex_0, capex_0]  # Create a list with the initial value

# Free Cash Flow
fcf_values = [fcf_0]  # Create a list with the initial value
for i in range(1, 6):  # Generate five additional values
    fcf_i = afterTax_values[i] - nwc_values[i] - capex_values[i]
    fcf_values.append(fcf_i)
    
# Present Value
presentValue_values = [presentValue_0]  # Create a list with the initial value
for i in range(1, 6):  # Generate five additional values
    presentValue_i = fcf_values[i] / ((1 + discount_rate)**(i))
    presentValue_values.append(presentValue_i)

presentValueSum = sum(presentValue_values)

# Perpetual Growth

In [82]:
longTermGrowth_rate = 0.04
fcf_n = fcf_values[-1] * (1 + longTermGrowth_rate)
terminalValue_n = fcf_n / (discount_rate - longTermGrowth_rate)
pv_of_tv = terminalValue_n / ((1 + discount_rate)**5)
net_debt = netDebt_0
dcf_value = presentValueSum + pv_of_tv - net_debt
share_outstanding = sharesOutstanding_0
equity_value = dcf_value / share_outstanding
print(equity_value)

55.309410560609706


# EBIDTA

In [87]:
tv_2026 = ebidta_multiple * ebidta_values[-1]
pv_of_tv2 = tv_2026 / ((1 + discount_rate)**(5))
enterprise_value = presentValueSum + pv_of_tv2 - net_debt
equity_value2 = enterprise_value / share_outstanding
print(equity_value2)

145.74138316666824


# Final Analysis

In [ ]:
df = pd.DataFrame(data=values, index=['EBIDTA', 'Earnings after DA', 'Applicable Tax Rate', 'After Tax',
'Net Working Capital', 'CAPEX', 'FCF', 'Discount Rate', 'Present Values', 'Annual Growth Rate', 'PV Sum'],
columns=['2021', '2022', '2023', '2024', '2025', '2026'])
df1 = pd.DataFrame(data=pp, index=['Long Term Growth Rate', '2026 FCF x (1+LTGR)', 'Terminal value 2026', 'PV of TV',
'DCF VALUE', 'Total Debt', 'Non Operating Assets', 'Net Debt'], columns=['Values'])
df2 = pd.DataFrame(data=eb, index=['EBITDA Multiple', 'Terminal Value in 2026', 'PV of TV', 'Enterprise Value'], columns=['Values'])

In [83]:
print('-----------------------------------------------------')
print('Determining Company Intrinsic Value for: ' , symbol)
print('Retrieving Data...')
print('-----------------------------------------------------\n')
print(tabulate(df, headers='keys', tablefmt='psql'))
print('\n-----------------------------------------------------')
print('Perpetual Growth Valuation')
print('Retrieving Data...')
print('-----------------------------------------------------\n')
print(tabulate(df1, headers='keys', tablefmt='psql'))
print('\n-----------------------------------------------------')
print('EBITDA Multiple Growth')
print('Retrieving Data...')
print('-----------------------------------------------------\n')

-----------------------------------------------------


NameError: name 'stock' is not defined